In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

# 1. Генерація даних
# Використовуємо параметри з вашого завдання
X, y = make_circles(n_samples=5000, noise=0.2, factor=0.5, random_state=1)

# # 2. Візуалізація вхідних даних
# plt.figure(figsize=(8, 6))
# # Малюємо точки класу 0 (червоні) та класу 1 (сині)
# plt.scatter(X[y == 0, 0], X[y == 0, 1], color='red', label='Class 0 (Outer Circle)', alpha=0.6)
# plt.scatter(X[y == 1, 0], X[y == 1, 1], color='blue', label='Class 1 (Inner Circle)', alpha=0.6)
# plt.title("Make Circles Dataset: Non-linear Structure")
# plt.xlabel("Feature 1")
# plt.ylabel("Feature 2")
# plt.legend()
# plt.grid(True)
# plt.show()

# 3. Розбиття даних (Train / Validation / Test) -> 60% / 20% / 20%

# Етап А: Відокремлюємо 60% на Train. Залишається 40% (temp) для Val+Test.
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

# Етап Б: Розбиваємо X_temp та y_temp (ті 40%, що залишилися) навпіл
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, 
    test_size=0.5, 
    random_state=42, 
    stratify=y_temp 
)

# Перевірка розмірностей
print(f"Загальна кількість: {X.shape[0]}")
print(f"Train set shape: {X_train.shape} ({X_train.shape[0]/X.shape[0]:.0%})")
print(f"Validation set shape: {X_val.shape} ({X_val.shape[0]/X.shape[0]:.0%})")
print(f"Test set shape: {X_test.shape} ({X_test.shape[0]/X.shape[0]:.0%})")

Загальна кількість: 5000
Train set shape: (3000, 2) (60%)
Validation set shape: (1000, 2) (20%)
Test set shape: (1000, 2) (20%)


In [4]:
# Copy this code into a new cell in your notebook

# Imports
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Models and Parameters
models = {
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.1, 1, 10, 100],
            'solver': ['liblinear', 'lbfgs']       }
    },
    'KNeighborsClassifier': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        }
    },
    'DecisionTreeClassifier': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'max_depth': [None, 5, 10, 20],
            'min_samples_split': [2, 5, 10],
            'criterion': ['gini', 'entropy']
        }
    },
    'SVC': {
        'model': SVC(random_state=42),
        'params': {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'rbf', 'poly'],
            'gamma': ['scale', 'auto']
        }
    }
}

# Tuning
best_estimators = {}
best_params = {}

for name, config in models.items():
    print(f"Tuning {name}...")
    search = HalvingGridSearchCV(
        config['model'],
        config['params'],
        factor=3,
        cv=5,
        random_state=42,
        n_jobs=-1
    )
    search.fit(X_train, y_train)
    
    print(f"Best Parameters for {name}: {search.best_params_}")
    print(f"Best Score: {search.best_score_:.4f}")
    best_estimators[name] = search.best_estimator_
    best_params[name] = search.best_params_
    print("-" * 30)

Tuning LogisticRegression...
Best Parameters for LogisticRegression: {'C': 1, 'solver': 'lbfgs'}
Best Score: 0.5027
------------------------------
Tuning KNeighborsClassifier...
Best Parameters for KNeighborsClassifier: {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'uniform'}
Best Score: 0.8835
------------------------------
Tuning DecisionTreeClassifier...
Best Parameters for DecisionTreeClassifier: {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2}
Best Score: 0.8698
------------------------------
Tuning SVC...
Best Parameters for SVC: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Best Score: 0.8948
------------------------------
